In [55]:
import numpy as np
import pandas as pd
import keras
from keras import Sequential
from keras.layers import LSTM, Dense, Activation, GRU, Bidirectional, Dropout
from keras.callbacks import ModelCheckpoint
from random import randint
import tensorflow as tf

In [56]:
# Read in the data into one single string named corpus. 
data = pd.read_csv('abba.csv')
df = pd.DataFrame(data)
corpus = df['lyrics'].str.cat(sep=' ')
df.head()
print(corpus)

My my
At Waterloo Napoleon did surrender
Oh yeah
And I have met my destiny in quite a similar way
The history book on the shelf
Is always repeating itself
Waterloo I was defeated, you won the war
Waterloo promise to love you for ever more
Waterloo couldn't escape if I wanted to
Waterloo knowing my fate is to be with you
Waterloo finally facing my Waterloo
My my
I tried to hold you back, but you were stronger
Oh yeah
And now it seems my only chance is giving up the fight
And how could I ever refuse
I feel like I win when I lose
Waterloo I was defeated, you won the war
Waterloo promise to love you for ever more
Waterloo couldn't escape if I wanted to
Waterloo knowing my fate is to be with you
Oh, oh Waterloo finally facing my Waterloo
So how could I ever refuse
I feel like I win when I lose
Waterloo couldn't escape if I wanted to
Waterloo knowing my fate is to be with you
Waterloo finally facing my Waterloo
Waterloo knowing my fate is to be with you
Oh, oh Waterloo finally facing my Wate

In [57]:
# Clean the data.
# Remove unicode characters, \r and other special characters.
# Save in file to check

import re

# Clean the data
cleaned_corpus = re.sub(r'[^\x00-\x7F]+', '', corpus)  # Remove unicode characters
cleaned_corpus = cleaned_corpus.replace('\r', '')  # Remove \r characters
cleaned_corpus = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_corpus)  # Remove other special characters

with open('cleaned_corpus.txt', 'w') as file:
    file.write(cleaned_corpus)

print("Data cleaned and saved to 'cleaned_corpus.txt'")





Data cleaned and saved to 'cleaned_corpus.txt'


In [58]:
# Create encoder and decoder dictionaries.
# each character is mapped to an integer and vice versa.
chars = sorted(list(set(corpus)))
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for c, i in enumerate(chars)}

In [59]:
# Define the sentence length and skip value
sentence_length = 20
skip = 1

# Initialize lists to hold the input sequences and the corresponding next characters
X_data = []
y_data = []

# Slice the corpus into semi-redundant sequences of 20 characters
for i in range(0, len(corpus) - sentence_length, skip):
    sentence = corpus[i:i + sentence_length]
    next_char = corpus[i + sentence_length]
    X_data.append([encoding[char] for char in sentence])
    y_data.append(encoding[next_char])

# Print the first few sequences to verify
print(X_data[:5], y_data[:5])

[[32, 71, 2, 59, 71, 1, 0, 20, 66, 2, 42, 47, 66, 51, 64, 58, 61, 61, 2, 33], [71, 2, 59, 71, 1, 0, 20, 66, 2, 42, 47, 66, 51, 64, 58, 61, 61, 2, 33, 47], [2, 59, 71, 1, 0, 20, 66, 2, 42, 47, 66, 51, 64, 58, 61, 61, 2, 33, 47, 62], [59, 71, 1, 0, 20, 66, 2, 42, 47, 66, 51, 64, 58, 61, 61, 2, 33, 47, 62, 61], [71, 1, 0, 20, 66, 2, 42, 47, 66, 51, 64, 58, 61, 61, 2, 33, 47, 62, 61, 58]] [47, 62, 61, 58, 51]


In [60]:
# simple check.
X_data[1], y_data[1]


([71, 2, 59, 71, 1, 0, 20, 66, 2, 42, 47, 66, 51, 64, 58, 61, 61, 2, 33, 47],
 62)

In [61]:
num_sentences = len(X_data)
print("Sliced our corpus into {0} sentences of length {1}".format(num_sentences, sentence_length))

Sliced our corpus into 217363 sentences of length 20


In [62]:
# now we need one-hot encoding
print("Vectorizing X and y...")
X = np.zeros((num_sentences, sentence_length, num_chars), dtype=bool)
y = np.zeros((num_sentences, num_chars), dtype=bool)
for i, sentence in enumerate(X_data):
    for t, encoded_char in enumerate(sentence):
        X[i, t, encoded_char] = 1
    y[i, y_data[i]] = 1

Vectorizing X and y...


In [63]:
# Define our model
print("Let's build model.")
model = Sequential()
model.add(LSTM(32, input_shape=(sentence_length, num_chars), return_sequences=True))
model.add(Bidirectional(LSTM(32, return_sequences=True)))  
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(num_chars))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary() 

Let's build model.


C:\Users\Pascal\Desktop\Projects\env\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                  │ (None, 20, 32)         │        15,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 20, 64)         │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 20, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 90)             │         5,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 90)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,258 (278.35 KB)

 Trainable params: 71,258 (278.35 KB)

 Non-trainable params: 0 (0.00 B)

In [65]:
## Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='loss', mode='min')

# Train the model for at least 20 epochs and save the best weights
history = model.fit(X, y, epochs=20, batch_size=128, callbacks=[checkpoint])

# Save the final model in the native Keras format
model.save('final_model.keras')

# Analyze the training history to check for overfitting
plt.plot(history.history['loss'], label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Epoch 1/20
1699/1699 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 3.1538

1699/1699 ━━━━━━━━━━━━━━━━━━━━ 151s 73ms/step - loss: 3.1536
Epoch 2/20
1698/1699 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 2.4172

1699/1699 ━━━━━━━━━━━━━━━━━━━━ 111s 64ms/step - loss: 2.4171
Epoch 3/20
1699/1699 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.2537

1699/1699 ━━━━━━━━━━━━━━━━━━━━ 109s 64ms/step - loss: 2.2537
Epoch 4/20
1699/1699 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 2.1319

1699/1699 ━━━━━━━━━━━━━━━━━━━━ 109s 64ms/step - loss: 2.1319
Epoch 5/20
1699/1699 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 2.0391

...

1699/1699 ━━━━━━━━━━━━━━━━━━━━ 133s 78ms/step - loss: 1.6071
Epoch 18/20
1699/1699 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 1.5919

1699/1699 ━━━━━━━━━━━━━━━━━━━━ 142s 83ms/step - loss: 1.5919
Epoch 19/20
1698/1699 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 1.5702

1699/1699 ━━━━━━━━━━━━━━━━━━━━ 117s 68ms/step - loss: 1.5703
Epoch 20/20
1699/1699 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 1.5629

1699/1699 ━━━━━━━━━━━━━━━━━━━━ 102s 60ms/step - loss: 1.5629


NameError: name 'plt' is not defined

In [52]:
model = Sequential()
model.add(LSTM(32, input_shape=(sentence_length, num_chars), return_sequences=True))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(num_chars))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='loss', mode='min')

# Train the model for at least 20 epochs and save the best weights
history = model.fit(X, y, epochs=20, batch_size=128, callbacks=[checkpoint])

# Save the final model in the native Keras format
model.save('final_model.keras')

# Analyze the training history to check for overfitting
plt.plot(history.history['loss'], label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

AttributeError: module 'keras.src.callbacks' has no attribute 'set_model'